In [ ]:
import json

import mysql.connector
import json
import pymysql



# Establishing a connection to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="layakishore",
    database ="USERS_DATA")


if conn.is_connected():
    print("Connection to MySQL server is working.")
else:
    print("Connection to MySQL server is not established.")
mycur= conn.cursor()

database_name = "USERS_DATA"

try:
   # mycur.execute(f"CREATE DATABASE {database_name}")
    print(f"Database '{database_name}' created successfully")
except mysql.connector.Error as err:
    print(f"Error: {err}")

create_table_query = """
CREATE TABLE IF NOT EXISTS users4 (
    id VARCHAR(20) PRIMARY KEY,
    name VARCHAR(255),
    screen_name VARCHAR(255),
    location VARCHAR(255),
    verified BOOLEAN,
    followers_count INT,
    statuses_count INT,
    friends_count INT
)
"""

# Execute the query to create the table
#mycur.execute(create_table_query)

mycur.execute("describe Users4")
for X in mycur:
    print(X)

connection = pymysql.connect(host='localhost',
                             user='root',
                             password='layakishore',
                             database='USERS_DATA',
                             cursorclass=pymysql.cursors.DictCursor)

from elasticsearch import Elasticsearch

# Define Elasticsearch Cloud credentials
cloud_id = "8231a925bb0f402aa30be4fac7ebcd97:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ1OTg4NzE5MWM5ODc0YjZmODc4ZjA1MmQ2NmI2M2Y1NyQwZmQ2ZWU0YWU4NDQ0NDQ4YjQ1NzI2NmQxMzlmOWI0Mw=="
cloud_auth = ("elastic", "KBDw46EL90pogZZOdcLrmyZ4")  # Credentials for accessing Elasticsearch Cloud

# Create Elasticsearch client with the specified parameters

es = Elasticsearch(cloud_id=cloud_id, basic_auth=cloud_auth)

# Check if the connection is successful
if es.ping():
    print("Connected to Elasticsearch Cloud")
else:
    print("Connection failed")



index_name = "tweets_final_index"



# Define the mapping
mapping = {
    "settings": {
        "number_of_shards": 9,
        "number_of_replicas": 0,
        "index.mapping.total_fields.limit": 2000
    },
    "mappings": {
        "properties": {
            "created_at": {"type": "date", "format": "EEE MMM dd HH:mm:ss Z yyyy"},
            "id": {"type": "long"},
            "id_str": {"type": "keyword"},
            "text": {"type": "text", "fielddata": True},
            "user": {
                "properties": {
                    "id": {"type": "long"},
                    "id_str": {"type": "keyword"},
                    "name": {"type": "text"},
                    "screen_name": {"type": "keyword"},
                  #  "dynamic": "false"
                }
            },
            "retweeted_status": {
                "properties": {
                    "created_at": {"type": "date", "format": "EEE MMM dd HH:mm:ss Z yyyy"},
                    "id": {"type": "long"},
                    "id_str": {"type": "keyword"},
                    "text": {"type": "text", "fielddata": True},
                    "user": {
                        "properties": {
                            "id": {"type": "long"},
                            "id_str": {"type": "keyword"},
                            "name": {"type": "text"},
                            "screen_name": {"type": "keyword"}
                        }
                    },
                    "retweet_count": {"type": "integer"},
                    "favorite_count": {"type": "integer"},
                    "favorited": {"type": "boolean"},
                    "retweeted": {"type": "boolean"},
                    "possibly_sensitive": {"type": "boolean"}
                }
            },
            "retweet_count": {"type": "integer"},
            "favorite_count": {"type": "integer"},
            "favorited": {"type": "boolean"},
            "retweeted": {"type": "boolean"},
            "possibly_sensitive": {"type": "boolean"}
        }
    }
}

# Create the index with the mapping
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping, ignore=400)  # ignore 400 already exists code


index_body_tweetids = {
    "settings": {
        "number_of_shards": 5,
        "number_of_replicas": 0,
        "index.mapping.total_fields.limit": 1000,  # Limiting the number of fields
    },
    "mappings": {
        "properties": {
            "tweet_id": {"type": "keyword"},  # Using keyword for exact matches
        }
    }
}

# Create the index
index_name_tweetids = "tweet_ids_final"
if not es.indices.exists(index=index_name_tweetids):
    es.indices.create(index=index_name_tweetids, body=index_body_tweetids)
    print(f"Index '{index_name_tweetids}' created.")
else:
    print(f"Index '{index_name_tweetids}' already exists.")
'''
new index for retweets by this user
new index for likes by this user
new index for recent searches by this user
new index for search?

Relavance Order
Total runtime of the program is 750.3318951129913 seconds

search by string, hashtag,  and user at the minimum. Allow for a time range in these searches. You will also need some notion of relevance to rank the search results. 

Show the meta-data for each tweet such as (a) the author of the tweet,  (b) when it was tweeted, (c) how many times it was retweeted. If the user of the search application clicks on the number of times it was retweeted, show the information about the retweets (who retweeted, when etc.).

If user of the search application clicks on the author, show other tweets by that author or retweets by that author (if available). 

 How you store your information will impact the response times of your query. You must discuss this (in your presentation and report). 

Allow for a time range in these searches?
'''
insert_sql = "INSERT INTO Users4 (id, name, screen_name, location, verified, followers_count, statuses_count, friends_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

check_sql = "SELECT COUNT(*) FROM Users4 WHERE id = %s"

def preparedTweet(tweet,user_info,retweeted_status,retweeted_user_info,isretweet,engagement_score1=0,engagement_score2=0):
    prepared_tweet = {
                "created_at": tweet.get("created_at"),
                "id": tweet.get("id"),
                "id_str": tweet.get("id_str"),
                "text": tweet.get("text"),
                "user": {
                    "id": user_info.get("id"),
                    "id_str": user_info.get("id_str"),
                    "name": user_info.get("name"),
                    "screen_name": user_info.get("screen_name")
                },
                "isretweet":isretweet,
                "retweeted_status": {
                    "created_at": retweeted_status.get("created_at"),
                    "id": retweeted_status.get("id"),
                    "id_str": retweeted_status.get("id_str"),
                    "text": retweeted_status.get("text"),
                    "user": {
                        "id": retweeted_user_info.get("id"),
                        "id_str": retweeted_user_info.get("id_str"),
                        "name": retweeted_user_info.get("name"),
                        "screen_name": retweeted_user_info.get("screen_name")
                    },
                    "retweet_count": retweeted_status.get("retweet_count"),
                    "favorite_count": retweeted_status.get("favorite_count"),
                    "favorited": retweeted_status.get("favorited"),
                    "retweeted": retweeted_status.get("retweeted"),
                    "possibly_sensitive": retweeted_status.get("possibly_sensitive"),
                    "engagement_score1": engagement_score1
                } if retweeted_status else {},
                "retweet_count": tweet.get("retweet_count"),
                "favorite_count": tweet.get("favorite_count"),
                "favorited": tweet.get("favorited"),
                "retweeted": tweet.get("retweeted"),
                "possibly_sensitive": tweet.get("possibly_sensitive"),
                "engagement_score2": engagement_score2
            }
    
    return prepared_tweet

def index_tweet(tweet):
    res = es.index(index=index_name, id=tweet["id"], document=tweet)
    #print(f"Indexed Tweet ID: {tweet['id']} Status: {res['result']}")


def insert_user_mysql(user):
    cursor.execute(insert_sql, (
                    user['id_str'],
                    user['name'],
                    user['screen_name'],
                    user['location'],
                    user['verified'],
                    user['followers_count'],
                    user['statuses_count'],
                    user['friends_count']
                ))
                # Commit the transaction
  #  print("insert successfull")
    return 

import time
start_time = time.time()
# Read the file and index each tweet

with connection.cursor() as cursor:
    with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2", 'r') as file:
        for line in file:

            if not line.strip():
                continue

            tweet = json.loads(line.strip())
            tweet_id = tweet.get("id_str")

            if es.exists(index=index_name_tweetids, id=tweet_id):
                    print(f"Tweet ID {tweet_id} already exists.")
                    continue

            es.index(index=index_name_tweetids, id=tweet_id, document={"tweet_id": tweet_id})

            user_info = tweet.get("user", {})
            retweeted_status = tweet.get("retweeted_status", {})
            retweeted_user_info = retweeted_status.get("user", {}) if retweeted_status else {}



            if ( tweet['text'].startswith('RT') ):

                #check if user is there or not
                user_id=user_info.get("id_str")
                cursor.execute(check_sql, (user_id,))
                result = cursor.fetchone()
                if result['COUNT(*)'] == 0:
                    insert_user_mysql(user_info)
                    connection.commit()
               #     print("commit success")
                    #if not load user into user index of es
                    # load or update user info in mysql
                    
                isretweet=True
                prepared_tweet=preparedTweet(tweet,user_info,retweeted_status,retweeted_user_info,isretweet)
                index_tweet(prepared_tweet)
                
                #check if Retweetuser is there or not
                if retweeted_status:
                   
                    tweet_id = retweeted_status.get("id_str")
                    if es.exists(index=index_name_tweetids, id=tweet_id):
                        print(f"Tweet ID {tweet_id} already exists.")
                        continue
                    else:
                        Ruser_id=retweeted_user_info.get("id_str")
                        cursor.execute(check_sql, (Ruser_id,))
                        result = cursor.fetchone()
                        if result['COUNT(*)'] == 0:
                            insert_user_mysql(retweeted_user_info)
                            connection.commit()
                        #    print("commit success")
                        #create a new original tweet for the retweet
                        isretweet=False
                        rc=retweeted_status.get("retweet_count")
                        lc=retweeted_status.get("favorite_count")
                        engagement_score2=(rc*rc)+lc
                        prepared_tweet=preparedTweet(tweet=retweeted_status,user_info=retweeted_user_info,retweeted_status={},retweeted_user_info={},isretweet=isretweet,engagement_score1=0,engagement_score2=engagement_score2)
                 #       print(prepared_tweet["engagement_score2"])
                        index_tweet(prepared_tweet)
                        print("Creating Original tweet")
                 
                else:
                    print(f"abnormal tweet ID {tweet_id}")
                    continue



            else:

                #check if user is there or not
                user_id=user_info.get("id_str")
                cursor.execute(check_sql, (user_id,))
                result = cursor.fetchone()
                if result['COUNT(*)'] == 0:
                    insert_user_mysql(user_info)
                    connection.commit()
              #      print("commit success")
                    #if not load user into user index of es
                    # load or update user info in mysql

                #load info into new table, tweetid, userid,screen_name, text, creationtime, retweet_count, Fav_count, retweeted, favourited, 
                #isretweet=false, Rtweetid, Ruserid,screen_name, Rcreationtime, retweet_count, Fav_count, retweeted, favourited

                #load info into main table
                isretweet=False
                rc=tweet.get("retweet_count")
                lc=tweet.get("favorite_count")
                engagement_score2=(rc*rc)+lc
                prepared_tweet=preparedTweet(tweet=tweet,user_info=user_info,retweeted_status=retweeted_status,retweeted_user_info=retweeted_user_info,isretweet=isretweet,engagement_score1=0,engagement_score2=engagement_score2)
               # print(prepared_tweet["engagement_score2"])
                index_tweet(prepared_tweet)





            #es.index(index=index_name_tweetids, id=tweet_id, document={"tweet_id": tweet_id, "user_id": user_id})

connection.close()
end_time = time.time()  # Capture end time
print(f"Total runtime of the program is {end_time - start_time} seconds")
print("All tweets have been indexed.")

import time
start_time = time.time()
# Read the file and index each tweet

with connection.cursor() as cursor:
    with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-3", 'r') as file:
        for line in file:

            if not line.strip():
                continue

            tweet = json.loads(line.strip())
            tweet_id = tweet.get("id_str")

            if es.exists(index=index_name_tweetids, id=tweet_id):
                    print(f"Tweet ID {tweet_id} already exists.")
                    continue

            es.index(index=index_name_tweetids, id=tweet_id, document={"tweet_id": tweet_id})

            user_info = tweet.get("user", {})
            retweeted_status = tweet.get("retweeted_status", {})
            retweeted_user_info = retweeted_status.get("user", {}) if retweeted_status else {}



            if ( tweet['text'].startswith('RT') ):

                #check if user is there or not
                user_id=user_info.get("id_str")
                cursor.execute(check_sql, (user_id,))
                result = cursor.fetchone()
                if result['COUNT(*)'] == 0:
                    insert_user_mysql(user_info)
                    connection.commit()
               #     print("commit success")
                    #if not load user into user index of es
                    # load or update user info in mysql
                    
                isretweet=True
                prepared_tweet=preparedTweet(tweet,user_info,retweeted_status,retweeted_user_info,isretweet)
                index_tweet(prepared_tweet)
                
                #check if Retweetuser is there or not
                if retweeted_status:
                   
                    tweet_id = retweeted_status.get("id_str")
                    if es.exists(index=index_name_tweetids, id=tweet_id):
                        print(f"Tweet ID {tweet_id} already exists.")
                        continue
                    else:
                        Ruser_id=retweeted_user_info.get("id_str")
                        cursor.execute(check_sql, (Ruser_id,))
                        result = cursor.fetchone()
                        if result['COUNT(*)'] == 0:
                            insert_user_mysql(retweeted_user_info)
                            connection.commit()
                        #    print("commit success")
                        #create a new original tweet for the retweet
                        isretweet=False
                        rc=retweeted_status.get("retweet_count")
                        lc=retweeted_status.get("favorite_count")
                        engagement_score2=(rc*rc)+lc
                        prepared_tweet=preparedTweet(tweet=retweeted_status,user_info=retweeted_user_info,retweeted_status={},retweeted_user_info={},isretweet=isretweet,engagement_score1=0,engagement_score2=engagement_score2)
                 #       print(prepared_tweet["engagement_score2"])
                        index_tweet(prepared_tweet)
                        print("Creating Original tweet")
                 
                else:
                    print(f"abnormal tweet ID {tweet_id}")
                    continue



            else:

                #check if user is there or not
                user_id=user_info.get("id_str")
                cursor.execute(check_sql, (user_id,))
                result = cursor.fetchone()
                if result['COUNT(*)'] == 0:
                    insert_user_mysql(user_info)
                    connection.commit()
              #      print("commit success")
                    #if not load user into user index of es
                    # load or update user info in mysql

                #load info into new table, tweetid, userid,screen_name, text, creationtime, retweet_count, Fav_count, retweeted, favourited, 
                #isretweet=false, Rtweetid, Ruserid,screen_name, Rcreationtime, retweet_count, Fav_count, retweeted, favourited

                #load info into main table
                isretweet=False
                rc=tweet.get("retweet_count")
                lc=tweet.get("favorite_count")
                engagement_score2=(rc*rc)+lc
                prepared_tweet=preparedTweet(tweet=tweet,user_info=user_info,retweeted_status=retweeted_status,retweeted_user_info=retweeted_user_info,isretweet=isretweet,engagement_score1=0,engagement_score2=engagement_score2)
               # print(prepared_tweet["engagement_score2"])
                index_tweet(prepared_tweet)





            #es.index(index=index_name_tweetids, id=tweet_id, document={"tweet_id": tweet_id, "user_id": user_id})

connection.close()
end_time = time.time()  # Capture end time
print(f"Total runtime of the program is {end_time - start_time} seconds")
print("All tweets have been indexed.")